In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/ASD_Circuits/src')
from ASD_Circuits import *
from tabulate import tabulate
import csv

/home/jw3514/anaconda3/lib/python3.9/site-packages/statsmodels/compat/pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [3]:
def searchFil(text, DIR):
    RES = []
    for file in os.listdir(DIR):
        if text in file:
            RES.append(file)
    return RES

def LoadSA2(fname, DIR, topL=50):
    fin = open(DIR+fname, 'rt')
    max_cohe, max_bias, max_STRs = 0, 0, []
    for i, l in enumerate(fin):
        if i > topL:
            break
        l = l.strip().split()
        cohe = float(l[0])
        bias = float(l[1])
        STRs = l[2].split(",")
        if cohe > max_cohe:
            max_cohe = cohe
            max_bias = bias
            max_STRs = STRs
    return max_cohe, max_bias, max_STRs

In [4]:
InfoMat = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw_v3/InfoMat.Ipsi.csv", index_col=0)
adj_mat = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw_v3/WeightMat.Ipsi.csv", index_col=0)

In [5]:
size = 46
#biaslim_df = pd.read_csv(biaslim_dir + "biaslim.size.{}.txt".format(size), names=["size", "bias"])
ASD_DIR_SI = "/home/jw3514/Work/ASD_Circuits/dat/Circuits/SA/ASD_Pareto_Dec14_Info/"
ASD_DIR_Conn = "/home/jw3514/Work/ASD_Circuits/dat/Circuits/SA/ASD_Pareto_Dec14_Conn/"
ASD_BiasDF = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.csv", index_col="STR")
biaslim_df = pd.read_csv(
    "../dat/Circuits/SA/biaslims2/biaslim.size.46.top17.txt", names=["size", "bias"])
biaslim_df = biaslim_df.sort_values("bias", ascending=False)

In [13]:
f = open("ASD.SA.Circuits.Size46.csv", 'wt')
writer = csv.writer(f) 
writer.writerow(["idx", "Bias", "SI_score", "Conn_score", "gain STRs", "lost STRs", "STRs"])

55

In [14]:
idx = 0
top_STRs = ASD_BiasDF.head(size).index.values
scoreSI = ScoreCircuit_SI_Joint(top_STRs, InfoMat)
scoreCO = ScoreCircuit_NEdges(top_STRs, adj_mat)
print(scoreSI, scoreCO)
print(RegionDistributionsList(top_STRs))
writer.writerow([idx, ASD_BiasDF.loc[top_STRs, "EFFECT"].mean(), scoreSI, scoreCO, 
                "", "", ";".join(top_STRs)])


0.504659027153166 0.12098298676748583
Isocortex	18	Orbital_area_lateral_part; Prelimbic_area; Primary_somatosensory_area_lower_limb; Orbital_area_ventrolateral_part; Infralimbic_area; Anteromedial_visual_area; Retrosplenial_area_lateral_agranular_part; Primary_visual_area; Primary_motor_area; Secondary_motor_area; Visceral_area; Anterior_cingulate_area_dorsal_part; Posterior_parietal_association_areas; Lateral_visual_area; Agranular_insular_area_posterior_part; Primary_somatosensory_area_trunk; Gustatory_areas; Agranular_insular_area_ventral_part
Hippocampus	6	Dentate_gyrus; Field_CA2; Subiculum_ventral_part; Field_CA1; Field_CA3; Parasubiculum
Cortical_subplate	3	Claustrum; Endopiriform_nucleus_dorsal_part; Endopiriform_nucleus_ventral_part
Pallidum	1	Bed_nuclei_of_the_stria_terminalis
Midbrain	1	Anterior_pretectal_nucleus
Striatum	4	Nucleus_accumbens; Caudoputamen; Lateral_septal_nucleus_ventral_part; Lateral_septal_nucleus_rostral_rostroventral_part
Olfactory_areas	2	Anterior_olfact

1254

In [15]:
cc = 0
PrevSet = set(top_STRs)
for i, row in biaslim_df.iterrows():
    #print(row["fname"], DIR)
    idx += 1
    fil = searchFil("keepN_{}-minbias_{}.txt".format(size, row["bias"]), ASD_DIR_SI)[0]
    cohe, real_minbias, STRs = LoadSA2(fil, ASD_DIR_SI)
    score = ScoreCircuit_SI_Joint(STRs, InfoMat)
    score2 = ScoreCircuit_NEdges(STRs, adj_mat)
    add_strs = list(set(STRs).difference(PrevSet))
    lost_strs = list(set(PrevSet).difference(STRs))
    print(row["bias"], round(real_minbias, 3), round(score, 3),
          len(set(STRs).difference(top_STRs)))
    print("add", add_strs)
    print("drop", lost_strs)
    PrevSet = set(STRs)
    cc += 1
    if cc > 16:
        break
        
    if row["bias"] == 0.37:
        List2Fil(STRs, "Circuits.Info.Size46.Bias0.37.txt")
    if row["bias"] == 0.36:
        List2Fil(STRs, "Circuits.Info.Size46.Bias0.36.txt")
    
    writer.writerow([idx, real_minbias, score, score2, 
                ";".join(add_strs), ";".join(lost_strs), ";".join(STRs)])
    print()
f.close()

0.38 0.38 0.56 3
add ['Orbital_area_medial_part', 'Subiculum_dorsal_part', 'Supplemental_somatosensory_area']
drop ['Subparafascicular_nucleus_magnocellular_part', 'Endopiriform_nucleus_ventral_part', 'Intercalated_amygdalar_nucleus']

0.375 0.375 0.649 5
add ['Basolateral_amygdalar_nucleus', 'Entorhinal_area_lateral_part', 'Piriform_area']
drop ['Parasubiculum', 'Supplemental_somatosensory_area', 'Gustatory_areas']

0.37 0.37 0.701 7
add ['Anterior_cingulate_area_ventral_part', 'Anteromedial_nucleus_dorsal_part', 'Gustatory_areas']
drop ['Field_CA3', 'Visceral_area', 'Lateral_septal_nucleus_ventral_part']

0.365 0.365 0.729 8
add ['Field_CA3', 'Taenia_tecta']
drop ['Dentate_gyrus', 'Field_CA1']

0.36 0.36 0.762 10
add ['Primary_somatosensory_area_barrel_field', 'Supplemental_somatosensory_area', 'Ventral_medial_nucleus_of_the_thalamus']
drop ['Field_CA3', 'Taenia_tecta', 'Field_CA2']

0.355 0.355 0.791 12
add ['Taenia_tecta', 'Frontal_pole_cerebral_cortex']
drop ['Agranular_insular_ar

In [23]:
cc = 0
PrevSet = set(top_STRs)
for i, row in biaslim_df.iterrows():
    #print(row["fname"], DIR)

    fil = searchFil("keepN_{}-minbias_{}.txt".format(size, row["bias"]), ASD_DIR_Conn)[0]
    cohe, real_minbias, STRs = LoadSA2(fil, ASD_DIR_Conn)
    score = ScoreCircuit_NEdges(STRs, adj_mat)
    print(row["bias"], round(real_minbias, 3), round(score, 3),
          len(set(STRs).difference(top_STRs)))
    print("add", set(STRs).difference(PrevSet))
    print("drop", set(PrevSet).difference(STRs))
    PrevSet = set(STRs)
    cc += 1
    if cc > 16:
        break
        
    if row["bias"] == 0.37:
        List2Fil(STRs, "Circuits.Info.Size46.Bias0.37.txt")
    if row["bias"] == 0.36:
        List2Fil(STRs, "Circuits.Info.Size46.Bias0.36.txt")
        
    print()

0.38 0.38 0.138 3
add {'Subiculum_dorsal_part', 'Piriform_area', 'Orbital_area_medial_part'}
drop {'Subparafascicular_nucleus_magnocellular_part', 'Intercalated_amygdalar_nucleus', 'Endopiriform_nucleus_ventral_part'}

0.375 0.375 0.159 5
add {'Supplemental_somatosensory_area', 'Entorhinal_area_lateral_part'}
drop {'Lateral_septal_nucleus_ventral_part', 'Parasubiculum'}

0.37 0.37 0.171 7
add {'Anteromedial_nucleus_dorsal_part', 'Basolateral_amygdalar_nucleus'}
drop {'Field_CA1', 'Visceral_area'}

0.365 0.365 0.179 9
add {'Primary_somatosensory_area_barrel_field', 'Frontal_pole_cerebral_cortex', 'Anterolateral_visual_area'}
drop {'Lateral_septal_nucleus_rostral_rostroventral_part', 'Basomedial_amygdalar_nucleus', 'Anteromedial_nucleus_dorsal_part'}

0.36 0.36 0.186 10
add {'Basomedial_amygdalar_nucleus', 'Taenia_tecta', 'Lateral_septal_nucleus_rostral_rostroventral_part', 'Anterior_cingulate_area_ventral_part'}
drop {'Dentate_gyrus', 'Field_CA3', 'Field_CA2', 'Anterolateral_visual_area